In [1]:
import numpy as np
import sys
import hrr_common.utils as cu
import hrr_common as hrr_cm
import hrr_cobot_robot as hrr_rob
import spatialmath as sm
import rospy

### Save arrays in dictionaries

one_infront_A = np.load('rack_positions/1_infront_A.npy')
one_infront_q = np.load('rack_positions/1_infront_q.npy')
one_pickup_A = np.load('rack_positions/1_pickup_A.npy')
one_pickup_q = np.load('rack_positions/1_pickup_q.npy')

gripper_at_pos1_poses = {
    "pickup":  {"q": one_pickup_q, "A": one_pickup_A},
    "infront": {"q": one_infront_q, "A": one_infront_q}
}

np.save("rack_positions/gripper_at_pos1_poses",gripper_at_pos1_poses,allow_pickle=True)

infront_A = np.load('rack_positions/6_infront_A.npy')
infront_q = np.load('rack_positions/6_infront_q.npy')
pickup_A = np.load('rack_positions/6_pickup_A.npy')
pickup_q = np.load('rack_positions/6_pickup_q.npy')
inbetween0_A = np.load('rack_positions/6_inbetween0_A.npy')
inbetween0_q = np.load('rack_positions/6_inbetween0_q.npy')
inbetween1_A = np.load('rack_positions/6_inbetween1_A.npy')
inbetween1_q = np.load('rack_positions/6_inbetween1_q.npy')
pre_pose_pickup_A = np.load('rack_positions/6_pre_pose_pickup_A.npy')
pre_pose_pickup_q = np.load('rack_positions/6_pre_pose_pickup_q.npy')

screwdriver_at_pos6_poses = {
    "pickup":  {"q": pickup_q, "A": pickup_A},
    "infront": {"q": infront_q, "A": infront_A},
    "inbetween0": {"q": inbetween0_q, "A": inbetween0_A},
    "inbetween1": {"q": inbetween1_q, "A": inbetween1_A},
    "pre_pose_pickup": {"q": pre_pose_pickup_q, "A": pre_pose_pickup_A}
}

np.save("rack_positions/screwdriver_at_pos6_poses",screwdriver_at_pos6_poses,allow_pickle=True)

len(vacuumPoses)

gripper_poses = np.load("rack_positions/gripper_at_pos1_poses.npy",allow_pickle=True).item()

### Tool change routine: picking a new tool with empty hand

In [ ]:
#Switch Case for type of tool to be picked up --> load correct poses
#For example in case of WSG50:
poses = np.load("rack_positions/gripper_at_pos1_poses.npy",allow_pickle=True).item()

#Check if tool changer open, otherwise give back error and stop. If open, continue:

#Go to the correct starting pose (q_calib with joint1 +/- pi/2)

#If there is a pre_pose_pickup, go there; else go to infront

#Go to pickup (with slow speed!)

#Close Tool changer

#If there is inbetween0, go there

#If there is inbetween1, go there

#Go to infront

#Go to starting pose

### Tool change routine: place a tool into the rack

In [ ]:
#Switch Case for type of tool to be picked up --> load correct poses
#For example in case of WSG50:
poses = np.load("rack_positions/gripper_at_pos1_poses.npy",allow_pickle=True).item()

#Check if tool changer closed, otherwise give back error and stop. If open, continue:

#Go to the correct starting pose (q_calib with joint1 +/- pi/2)

#go to infront

#if there is inbetween1, go there

#if there is inbetween0, go there

#Go to pickup

#Open Tool changer

#If there is pre_pose pickup, go there; else go to infront

#Go to the correct starting pose

### End of Testing array manipulation

In [ ]:
#Copy paste here the proper ROSIP and ROSENV setup

In [ ]:
rospy.init_node('toolchange_dev')

In [ ]:
hrr_rob.load_default_parameters("/hrr_cobot")
cobot = hrr_rob.HrrCobotControl.from_ros()
print(cobot)

In [ ]:
#Routine to open and close tool changer
from comau_msgs.srv import *
setPin = rospy.ServiceProxy("/hrr_cobot/set_digital_io",SetIO)

In [ ]:
#Move to pose with speed as input
def cobot_movetopose(posee, v_maximale):
    cobot.move_to_pose(posee, err_gain = None, v_max = v_maximale) #start_pose, cobot.FK(cobot.q_calib)
    T = int(100 * cobot.hz)
    for t in trange(T):
        if cobot.state is None:
            rospy.loginfo(f"reached goal pose at step {t + 1} / {T}")
            break
        elif cobot.state == "error":
            rospy.logerror(f"robot in ERROR state")
        cobot.update()

## About this notebook
Position 1-6 are defined as follows: In the current setup, the robot points towards the disassembly table for joint1-angle=0. If the robot turns to its left (negative joint1-angle), it turns towards position 1, then 2, then 3. Starting from zero joint1-angle, if the robot turns towards its right (positive joint1-angle), it turns towards position 4, then 5, then 6 (then 3, 2, 1 if it keeps turning).

It is fixed here which tool is at which position, since the path of approach is different for (almost) each tool.

### The pickup procedure for position number p is as follows: 

0. Assume the cobot has no tool attached, toolchanger open and the cobot is at cobot.FK(cobot.q_calib) or similiar (joint1 might be different)
1. Set joint1 = -pi/2 for p = 1,2,3 and joint1 = +pi/2 for p = 4,5,6, call that pose the starting pose.
2. The cobot goes to the pickup pose (saved as p_pickup_q and p_pickup_A) and closes the toolchanger
3. The cobot goes to a pose infront of the rack slot from which it is safe to go to the starting pose (saved as p_infront_q and p_infront_A). Possibly, there are waypoint-poses to get infront of the rack safely (saved as p_inbetweenX_q and p_inbetweenX_A, where X is a number from 0 to N which the cobot passes through from low to high)
4. The cobot goes to the starting pose
5. (The cobot goes to the calibration pose)

### The remove procedure for tool to be stored at position number p is as follows:
0. Assume the cobot has a tool attached, toolchanger closed and the cobot is at cobot.FK(cobot.q_calib) or similiar (joint1 might be different)
1. Set joint1 = -pi/2 for p = 1,2,3 and joint1 = +pi/2 for p = 4,5,6, call that pose the starting pose.
2. The cobot goes to a pose infront of the rack slot (saved as p_infront_q and p_infront_A)
3. The cobot goes to the pickup pose (saved as p_pickup_q and p_pickup_A). Possibly, there are waypoint-poses to get into the rack safely (saved as p_inbetweenX_q and p_inbetweenX_A, where X is a number from 0 to N which the cobot passes through from high to low)
4. The toolchanger is opened and the cobot goes back to the starting pose. (should be safe, otherwise go through the waypoints again)
5. (The cobot goes to the calibration pose)


### Open tool changer

In [ ]:
setPin(pin=5, state=True)

### Close tool changer

In [ ]:
setPin(pin=5, state=False)

## Saving the poses

In [ ]:
starting_pose_123 = cobot.q
starting_pose_456 = cobot.q
starting_pose_123 = cobot.q[0] = -np.pi/2
starting_pose_456 = cobot.q[0] = np.pi/2

### Position 1 - WSG50

In [ ]:
#Go with robot manually to pickup position, then
cobot.update_tf()
np.save("rack_positions/1_pickup_q",cobot.q,allow_pickle=True)
np.save("rack_positions/1_pickup_A",cobot.T_B_E_robot.A,allow_pickle=True)

In [ ]:
#Close tool and go up a bit (positive z in base frame) to save a new waypoint
cobot.update_tf()
np.save("rack_positions/1_inbetween0_q",cobot.q,allow_pickle=True)
np.save("rack_positions/1_inbetween0_A",cobot.T_B_E_robot.A,allow_pickle=True)

In [ ]:
#Pull out the tool fully from the slot, at a safe position, save it
cobot.update_tf()
np.save("rack_positions/1_infront_q",cobot.q,allow_pickle=True)
np.save("rack_positions/1_infront_A",cobot.T_B_E_robot.A,allow_pickle=True)

In [ ]:
# Check if the way to starting pose is safe
cobot_movetopose(starting_pose_123, 0.01)

### Position 2 - Vacuum Gripper

In [ ]:
#Go with robot to pickup position, then
cobot.update_tf()
np.save("rack_positions/2_pickup_q",cobot.q,allow_pickle=True)
np.save("rack_positions/2_pickup_A",cobot.T_B_E_robot.A,allow_pickle=True)

In [ ]:
#Intermediate waypoint, skip if not needed, add more if needed
cobot.update_tf()
np.save("rack_positions/2_inbetween0_q",cobot.q,allow_pickle=True)
np.save("rack_positions/2_inbetween0_A",cobot.T_B_E_robot.A,allow_pickle=True)

In [ ]:
#Pull out the tool fully from the slot, at a safe position, save it
cobot.update_tf()
np.save("rack_positions/2_infront_q",cobot.q,allow_pickle=True)
np.save("rack_positions/2_infront_A",cobot.T_B_E_robot.A,allow_pickle=True)

In [ ]:
# Check if the way to starting pose is safe
cobot_movetopose(starting_pose_123, 0.01)

### Position 3 - Shaft grinder

In [ ]:
#Go with robot to pickup position, then
cobot.update_tf()
np.save("rack_positions/3_pickup_q",cobot.q,allow_pickle=True)
np.save("rack_positions/3_pickup_A",cobot.T_B_E_robot.A,allow_pickle=True)

In [ ]:
#Intermediate waypoint, skip if not needed, add more if needed
cobot.update_tf()
np.save("rack_positions/3_inbetween0_q",cobot.q,allow_pickle=True)
np.save("rack_positions/3_inbetween0_A",cobot.T_B_E_robot.A,allow_pickle=True)

In [ ]:
#Pull out the tool fully from the slot, at a safe position, save it
cobot.update_tf()
np.save("rack_positions/3_infront_q",cobot.q,allow_pickle=True)
np.save("rack_positions/3_infront_A",cobot.T_B_E_robot.A,allow_pickle=True)

In [ ]:
# Check if the way to starting pose is safe
cobot_movetopose(starting_pose_123, 0.01)

### Position 4 - New Gripper? To be Decided

### Position 5 - Empty

### Position 6  - Screw driver


In [ ]:
#Go with robot to pickup position, then
cobot.update_tf()
np.save("rack_positions/6_pickup_q",cobot.q,allow_pickle=True)
np.save("rack_positions/6_pickup_A",cobot.T_B_E_robot.A,allow_pickle=True)

In [ ]:
#Intermediate waypoint, skip if not needed, add more if needed
cobot.update_tf()
np.save("rack_positions/6_inbetween0_q",cobot.q,allow_pickle=True)
np.save("rack_positions/6_inbetween0_A",cobot.T_B_E_robot.A,allow_pickle=True)

In [ ]:
#Pull out the tool fully from the slot, at a safe position, save it
cobot.update_tf()
np.save("rack_positions/6_infront_q",cobot.q,allow_pickle=True)
np.save("rack_positions/6_infront_A",cobot.T_B_E_robot.A,allow_pickle=True)

In [ ]:
# Check if the way to starting pose is safe
cobot_movetopose(starting_pose_456, 0.01)

## Tool Change Utility - Example: Dispose Shaft grinder and pickup Vacuum Gripper

### Start at calibration pose

In [ ]:
cobot_movetopose(cobot.FK(cobot.q_calib), 0.03)

### Put shaft grinder into grasp of toolchanger and close tool changer with the following command

In [ ]:
setPin(pin=5, state=False)

### Move to starting pose for tool position between 1-3

In [ ]:
cobot_movetopose(cobot.FK(starting_pose_123), 0.03)

### Load waypoints

In [ ]:
infront = np.load("rack_positions/3_infront_A.npy")
pickup = np.load("rack_positions/3_pickup_A.npy")


### Move infront of rack

In [ ]:
cobot_movetopose(infront, 0.03)

### Move to pickup pose

In [ ]:
cobot_movetopose(pickup, 0.03)

### Open tool changer

In [ ]:
setPin(pin=5, state=True)

### Go back to starting pose

In [ ]:
cobot_movetopose(cobot.FK(starting_pose_123), 0.03)

### Go to starting pose for vacuum gripper (1)

In [ ]:
cobot_movetopose(cobot.FK(starting_pose_123), 0.03)

### Load waypoints

In [ ]:
infront = np.load("rack_positions/1_infront_A.npy")
inter = np.load("rack_positions/1_inbetween0_A.npy")
pickup = np.load("rack_positions/1_pickup_A.npy")

### Go to pickup pose

In [ ]:
cobot_movetopose(pickup, 0.03)

### Close tool changer

In [ ]:
setPin(pin=5, state=False)

### Go to inbetween pose

In [ ]:
cobot_movetopose(inter, 0.03)

### Go to safe pose in front of rack

In [ ]:
cobot_movetopose(infront, 0.03)

### Go to starting pose

In [ ]:
cobot_movetopose(cobot.FK(starting_pose_123), 0.03)

### Go to calibration pose

In [ ]:
cobot_movetopose(cobot.FK(q.calib), 0.03)